In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/Dataset.zip" -d "/content/drive/MyDrive/Datasetnew"

Archive:  /content/drive/MyDrive/Dataset.zip
   creating: /content/drive/MyDrive/Datasetnew/Dataset/
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus1.jpg  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus1.xml  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus10.jpg  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus10.xml  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus100.jpg  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus100.xml  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus101.jpg  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus101.xml  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus102.jpg  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus102.xml  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus103.jpg  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus103.xml  
  inflating: /content/drive/MyDrive/Datasetnew/Dataset/bus104.jpg  
  inflating: /content

In [4]:
!pip install ultralytics matplotlib seaborn numpy

import os
import xml.etree.ElementTree as ET
import shutil
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

# Directories
DATASET_DIR = "/content/drive/MyDrive/Datasetnew/Dataset"
YOLO_DATA_DIR = "/content/yolo_dataset"

# Target classes
TARGET_CLASSES = {"car": 0, "motorbike": 1, "bus": 2}

# Create YOLO dataset structure
os.makedirs(os.path.join(YOLO_DATA_DIR, "images", "train"), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATA_DIR, "images", "val"), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATA_DIR, "labels", "train"), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATA_DIR, "labels", "val"), exist_ok=True)

# Debug: Check dataset directory
if not os.path.exists(DATASET_DIR):
    raise FileNotFoundError(f"Dataset directory {DATASET_DIR} not found")
print(f"Files in {DATASET_DIR}:", len(os.listdir(DATASET_DIR)), "files")

# Convert XML to YOLO format
class_counts = defaultdict(int)
image_count = 0
for file in os.listdir(DATASET_DIR):
    if file.endswith(".xml"):
        xml_path = os.path.join(DATASET_DIR, file)
        base_name = os.path.splitext(file)[0]
        img_file = base_name + ".jpg"
        img_path = os.path.join(DATASET_DIR, img_file)

        try:
            if not os.path.exists(img_path):
                print(f"Image not found: {img_path}")
                continue

            tree = ET.parse(xml_path)
            root = tree.getroot()
            size = root.find("size")
            if size is None:
                print(f"No size tag in {file}")
                continue
            width = float(size.find("width").text)
            height = float(size.find("height").text)
            if width == 0 or height == 0:
                print(f"Invalid image size in {file}")
                continue

            yolo_labels = []
            valid_image = False
            for obj in root.findall("object"):
                cls_name = obj.find("name").text.strip().lower()
                if cls_name in TARGET_CLASSES:
                    bndbox = obj.find("bndbox")
                    if bndbox is None:
                        print(f"No bndbox in object in {file}")
                        continue
                    xmin = float(bndbox.find("xmin").text)
                    ymin = float(bndbox.find("ymin").text)
                    xmax = float(bndbox.find("xmax").text)
                    ymax = float(bndbox.find("ymax").text)

                    x_center = (xmin + xmax) / 2 / width
                    y_center = (ymin + ymax) / 2 / height
                    box_width = (xmax - xmin) / width
                    box_height = (ymax - ymin) / height

                    yolo_labels.append(f"{TARGET_CLASSES[cls_name]} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")
                    class_counts[cls_name] += 1
                    valid_image = True

            if valid_image:
                split = "train" if np.random.rand() < 0.8 else "val"
                label_path = os.path.join(YOLO_DATA_DIR, "labels", split, f"{base_name}.txt")
                with open(label_path, "w") as f:
                    f.write("\n".join(yolo_labels))
                shutil.copy(img_path, os.path.join(YOLO_DATA_DIR, "images", split, img_file))
                image_count += 1
                print(f"Processed {file} -> {split} (Image {image_count})")

        except (ET.ParseError, ValueError) as e:
            print(f"Error processing {file}: {e}")
            continue

# Debug: Check YOLO dataset
print(f"Train images: {len(os.listdir(os.path.join(YOLO_DATA_DIR, 'images', 'train')))}")
print(f"Val images: {len(os.listdir(os.path.join(YOLO_DATA_DIR, 'images', 'val')))}")
print("Class counts:", dict(class_counts))

# Create data.yaml
data_yaml = f"""
train: {os.path.join(YOLO_DATA_DIR, 'images', 'train')}
val: {os.path.join(YOLO_DATA_DIR, 'images', 'val')}
nc: {len(TARGET_CLASSES)}
names: {list(TARGET_CLASSES.keys())}
"""
with open(os.path.join(YOLO_DATA_DIR, "data.yaml"), "w") as f:
    f.write(data_yaml)

# Load and train YOLOv8n
model = YOLO("yolov8n.pt")
results = model.train(
    data=os.path.join(YOLO_DATA_DIR, "data.yaml"),
    epochs=100,
    imgsz=416,
    batch=16,
    device=0,  # GPU
    patience=5,
    augment=True  # Enable data augmentation
)

# Evaluate model
metrics = model.val()

# Save model to Google Drive
model_path = "/content/drive/MyDrive/best.pt"
model.save(model_path)
print(f"Trained model saved at: {model_path}")

# Plot evaluation metrics
plt.figure(figsize=(10, 6))
metric_values = [
    metrics.box.map50,
    float(np.mean(metrics.box.mp) if hasattr(metrics.box, 'mp') else np.mean(metrics.box.p)),
    float(np.mean(metrics.box.mr) if hasattr(metrics.box, 'mr') else np.mean(metrics.box.r)),
    float(np.mean(metrics.box.f1))
]
sns.barplot(x=["mAP@0.5", "Precision", "Recall", "F1"], y=metric_values)
plt.title("Model Evaluation Metrics")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.savefig(os.path.join(YOLO_DATA_DIR, "evaluation_metrics.png"))
plt.close()

# Plot confusion matrix
cm = metrics.confusion_matrix.matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt=".0f", cmap="Blues", xticklabels=TARGET_CLASSES.keys(), yticklabels=TARGET_CLASSES.keys())
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(os.path.join(YOLO_DATA_DIR, "confusion_matrix.png"))
plt.close()

print(f"Metrics and plots saved in {YOLO_DATA_DIR}: evaluation_metrics.png, confusion_matrix.png")
print(f"Metrics: mAP@0.5={metrics.box.map50:.3f}, Precision={np.mean(metrics.box.mp) if hasattr(metrics.box, 'mp') else np.mean(metrics.box.p):.3f}, Recall={np.mean(metrics.box.mr) if hasattr(metrics.box, 'mr') else np.mean(metrics.box.r):.3f}, F1={np.mean(metrics.box.f1):.3f}")

Files in /content/drive/MyDrive/Datasetnew/Dataset: 4800 files
Processed bus1.xml -> train (Image 1)
Processed bus10.xml -> train (Image 2)
Processed bus100.xml -> train (Image 3)
Processed bus101.xml -> train (Image 4)
Processed bus102.xml -> train (Image 5)
Processed bus103.xml -> train (Image 6)
Processed bus104.xml -> train (Image 7)
Processed bus105.xml -> train (Image 8)
Processed bus106.xml -> train (Image 9)
Processed bus107.xml -> train (Image 10)
Processed bus108.xml -> val (Image 11)
Processed bus109.xml -> val (Image 12)
Processed bus11.xml -> train (Image 13)
Processed bus110.xml -> train (Image 14)
Processed bus111.xml -> val (Image 15)
Processed bus112.xml -> train (Image 16)
Processed bus113.xml -> val (Image 17)
Processed bus114.xml -> train (Image 18)
Processed bus115.xml -> train (Image 19)
Processed bus116.xml -> train (Image 20)
Processed bus117.xml -> val (Image 21)
Processed bus118.xml -> train (Image 22)
Processed bus119.xml -> train (Image 23)
Processed bus12.x

Ultralytics 8.3.181 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretrained=T

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     


  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256,

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2298.0±1612.5 MB/s, size: 788.4 KB)


...

                   all        474      11193      0.922      0.675      0.785      0.605
                   car        411       6203      0.942      0.704      0.832      0.655
             motorbike        353       1366       0.88      0.541      0.652      0.446
                   bus        387       3624      0.944      0.779      0.871      0.714
Speed: 0.1ms preprocess, 12.3ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to runs/detect/train
Ultralytics 8.3.181 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3246.2±330.0 MB/s, size: 2018.2 KB)


val: Scanning /content/yolo_dataset/labels/val.cache... 474 images, 0 backgrounds, 0 corrupt: 100%|██████████| 474/474 [00:00<?, ?it/s]

val: /content/yolo_dataset/images/val/bus111.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/bus113.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/car185.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike104.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike110.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike121.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike602.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike616.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike617.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike646.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike649.jpg: corrupt JPEG restored and saved
val: /content/yolo_dataset/images/val/motorbike658


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:20<00:00,  1.44it/s]


                   all        474      11193      0.918       0.68      0.786      0.608
                   car        411       6203      0.941      0.707      0.832      0.655
             motorbike        353       1366       0.88      0.542      0.656      0.451
                   bus        387       3624      0.933       0.79      0.871      0.717
Speed: 0.4ms preprocess, 11.2ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train2
Trained model saved at: /content/drive/MyDrive/best.pt
Metrics and plots saved in /content/yolo_dataset: evaluation_metrics.png, confusion_matrix.png
Metrics: mAP@0.5=0.786, Precision=0.918, Recall=0.680, F1=0.778


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Define output directory (modify as needed)
YOLO_DATA_DIR = "/content/drive/MyDrive"

# Ensure the output directory exists
os.makedirs(YOLO_DATA_DIR, exist_ok=True)

# Overall model metrics from provided data
overall_metrics = {
    "mAP@0.5": 0.786,
    "Precision": 0.918,
    "Recall": 0.680,
    "F1": 0.778
}

# Per-class metrics from provided data
classes = ["car", "motorbike", "bus"]
per_class_metrics = {
    "Precision": [0.941, 0.880, 0.933],
    "Recall": [0.707, 0.542, 0.790],
    "mAP@0.5": [0.832, 0.656, 0.871],
    "mAP@0.5:0.95": [0.655, 0.451, 0.717]
}

# Plot 1: Overall Model Evaluation Metrics (Bar Plot)
plt.figure(figsize=(10, 6))
metric_names = list(overall_metrics.keys())
metric_values = list(overall_metrics.values())
sns.barplot(x=metric_names, y=metric_values)
plt.title("Overall Model Evaluation Metrics")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.savefig(os.path.join(YOLO_DATA_DIR, "evaluation_metrics.png"))
plt.close()

# Plot 2: Per-Class Metrics (Grouped Bar Plot)
plt.figure(figsize=(12, 7))
x = np.arange(len(classes))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots()
ax.bar(x - width*1.5, per_class_metrics["Precision"], width, label="Precision", color="#4CAF50")
ax.bar(x - width*0.5, per_class_metrics["Recall"], width, label="Recall", color="#2196F3")
ax.bar(x + width*0.5, per_class_metrics["mAP@0.5"], width, label="mAP@0.5", color="#FF9800")
ax.bar(x + width*1.5, per_class_metrics["mAP@0.5:0.95"], width, label="mAP@0.5:0.95", color="#E91E63")

ax.set_xlabel("Class")
ax.set_ylabel("Score")
ax.set_title("Per-Class Evaluation Metrics")
ax.set_xticks(x)
ax.set_xticklabels(classes)
ax.legend()
ax.set_ylim(0, 1)
plt.savefig(os.path.join(YOLO_DATA_DIR, "per_class_metrics.png"))
plt.close()

# Plot 3: Confusion Matrix (Synthetic Example)
# Assuming 3 classes + background (common in YOLO)
# Since actual confusion matrix data is not provided, create a synthetic one
# Rows: True labels (car, motorbike, bus, background), Columns: Predicted labels
cm = np.array([
    [5800, 200, 100, 103],  # car (true) -> mostly car, some misclassified
    [150, 1100, 50, 66],    # motorbike (true)
    [200, 100, 3200, 124],  # bus (true)
    [50, 30, 40, 0]         # background (true)
])

# Class labels including background
target_classes = {0: "car", 1: "motorbike", 2: "bus", 3: "background"}
class_names = list(target_classes.values())

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt=".0f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix (Synthetic Example)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(os.path.join(YOLO_DATA_DIR, "confusion_matrix.png"))
plt.close()

print("Plots saved to:", YOLO_DATA_DIR)

Plots saved to: /content/drive/MyDrive


<Figure size 1200x700 with 0 Axes>

In [2]:
from google.colab import files
files.download("Training.ipynb")


FileNotFoundError: Cannot find file: Training.ipynb